# Fixed Point Step Functions.

Convert the algorithm for generating steps in the article ["Generate stepper-motor speed profiles in real time"](https://www.embedded.com/generate-stepper-motor-speed-profiles-in-real-time/) into a Fixed Point version. 

The key formula from the article is:

<img src="https://www.embedded.com/wp-content/uploads/media-1042023-0501feat1eq13.gif">

To make this function work in fixed point, we multiply both sides by a fixed point factor, $\alpha$:

1. $c_i = c_{i-1} - \frac{2c_{i-1}}{4n_i+1}$
1. $c_i\alpha = {\alpha}c_{i-1} - \alpha\frac{2c_{i-1}}{4n_i+1}$
1. ${\alpha}c_i = {\alpha}c_{i-1} - \frac{{\alpha}2c_{i-1}}{4n_i+1}$

Let $c_{\alpha} = {\alpha}c_{i-1}$

${\alpha}c_i = c_{\alpha} - \frac{2c_{\alpha}}{4n_i+1}$

With assignment notation:

$c_{\alpha} \leftarrow c_{\alpha} - \frac{2c_{\alpha}}{4n_i+1}$

In [1]:
from trajectory import SimSegment as ss
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
n, cn,ca = ss.initial_params(0,1000,1)
n, cn, ca, ca>>8

TypeError: initial_params() takes 1 positional argument but 3 were given

In [ ]:
from math import log, sqrt
TIMEBASE=1_000_000 # Counter an delays are measures in 1/TIMEBASE secs
FP_BITS = 8 # Number of fraction bits in fixed point representation of cn


def initial_params(v0, v1, t, timebase=None):
    """Set initial parameters, which are a bit different for the first step,
    to account for low torque. """

    # If n is positive, there is a non zero velocity and we are accelerating
    # If it is negative, there is a non zero velocity and we are decelerating

    timebase = timebase if timebase is not None else TIMEBASE

    v0 = float(v0)
    v1 = float(v1)

    if v0 == 0 and v1 == 0:
        # Going nowhere.
        n = 0
        cn = 0

    elif v0 == 0:
        # Starting from a stop, so need special
        # first delay.
        a = abs(v1) / t
        n = 0  # n will always be positive, so accelerating
        cn = 0.676 * sqrt(2.0 / a) * timebase  # c0 in Equation 15
        
        assert cn > 0

    elif v0 == v1:
        # Constant velocity.
        n = N_BIG
        cn = timebase / abs(v0)
    else:
        # Normal case, between two non-zero velocities
        a = abs(v1 - v0) / t
        n = (v0 * v0) / (2.0 * a)  # Equation 16
        cn = timebase / abs(v0)

        assert n >= 0

        # Need to put the sign back on n; n must be negative for deceleration
        if abs(v1) < abs(v0):
            n = -n

   
    # Fixed point version of cn
    ca = int(cn)<<FP_BITS
  
    n = int(n)

    return n, cn, ca

def next_params(n, cn):

    n += 1

    if cn != 0:
        cn = cn - ((2.0 * cn) / ((4.0 * n) + 1)) # Equation 13

    return int(n), cn

def next_params_xp(n, ca):
    """Fixed point version of the next_params function"""

    n += 1

    if n != 0:
         ca = (ca - ((ca << 1) / ((n << 2) + 1)))  # Equation 13

    
    return n, int(abs(ca))

def next_params_bres(n,ca):
    pass
    

def simulate(v0, v1, t):
    
    n, cn, ca = initial_params(v0,v1,t, timebase=TIMEBASE)
   
    yield n, cn, ca, ca>>FP_BITS
    
    x = abs(.5 * (v1 + v0) * t)
    
    for i in range(int(x)):
        _, ca = next_params_xp(n, ca)
        n, cn = next_params(n, cn)
        
        yield n,cn, ca, ca>>FP_BITS
     

df = pd.DataFrame(list(simulate(80,10000,1)), columns='n cn car ca'.split())
df['ca1'] = df.car - df.car.shift(-1) 
df['d'] = (df.cn-df.ca).abs()
df['cum_d'] = df.d.cumsum()
df['cum_cn'] = df.cn.cumsum()
df['pct_err'] = (df.d.cumsum() / df.cn.cumsum() * 100).round(3)
print(df.cn.max(), round(log(df.cn.max())/log(2)), df.pct_err.max() )
display(df.head())
df.tail()

    

In [ ]:

cn

In [ ]:
INTERRUPT_PERIOD = 5 #NUmber of TIMEBASE ticks for each update of the Bresenham line update. 
from fractions import Fraction

def next_params_bres(n,cn):
    f  = TIMEBASE/cn

    intr_per_step = cn/INTERRUPT_PERIOD
    r = round((1/intr_per_step)*1000,0)
    
    return intr_per_step, r
    
f = 10_000 + 121
n, cn, ca = initial_params(f, f+1,1)
s, r = next_params_bres(n,cn)
s, r

In [ ]:
%%time
errors = set()
for i in range(1,2000):
    for j in range(1,2000):
        r = float(i)/float(j)
        f = Fraction(i,j).limit_denominator(100)
        errors.add(abs(r-f))

In [ ]:
s = pd.Series(list(errors))
s.hist(bins=50)

In [ ]:
1/Fraction(13,17)

In [ ]:
%%time

nums = []
for i in range(1,256):
    for j in range(1,i):
        r = float(j)/float(i)
        f = Fraction(j,i).limit_denominator(100)
        nums.append([j,i,float(f), r])
        
s = pd.DataFrame(list(nums), columns = 'num denom rat real'.split())

s.hist(bins=50)

In [ ]:
s.describe()

In [ ]:
s.rat.nunique()

In [ ]:
5544/4000
